---
title: "强化学习专题 - 马尔可夫决策过程"
date: 2025-06-04T03:58:00+08:00
author: "Liu Zheng"
tags: ["笔记", "实验", "AI", "强化学习"]
categories: "实验笔记"
---

# 马尔可夫决策过程

## 1. 基本概念

马尔可夫决策过程（Markov Decision Process, MDP）是人工智能、运筹学和控制理论中用于对决策问题进行建模的数学框架。它提供了一种在随机环境中进行序列决策的方法，其中结果是部分随机的，部分受决策者控制。

马尔可夫决策过程通常由一个元组表示：$ MDP = (S, A, P, R, \gamma) $

其中：

1. 状态集 (States): $S = \{s_1, s_2, \dots, s_n\}$
2. 动作集 (Actions): $A = \{a_1, a_2, \dots, a_m\}$
3. 转移概率函数 (Transition Probability Function): $P: S \times A \times S \to [0, 1]$
    - $P(s' | s, a)$ 表示在状态 $s$ 执行动作 $a$ 后，转移到下一个状态 $s'$ 的概率。
    - 对于任意状态 $s \in S$ 和动作 $a \in A$，所有可能的下一个状态的概率之和必须为 1：$\sum_{s' \in S} P(s' | s, a) = 1$
4. 奖励函数 (Reward Function): $R: S \times A \times S \to \mathbb{R}$
    - $R(s, a, s')$ 表示在状态 $s$ 执行动作 $a$ 并转移到下一个状态 $s'$ 时获得的即时奖励。
    - 有时奖励函数也可以表示为 $R(s, a)$（在状态 $s$ 执行动作 $a$ 获得的奖励），或 $R(s')$（进入状态 $s'$ 获得的奖励）。最通用的形式是 $R(s, a, s')$。
5. 折扣因子 (Discount Factor): $\gamma \in [0, 1)$。通常要求 $\gamma < 1$ 以保证无限时间序列的总奖励收敛，它用于衡量未来奖励相对于当前奖励的重要性。

## 2. 术语定义：策略

策略（Policy）定义了智能体在每个状态下应该采取什么动作。

1. **确定性策略 (Deterministic Policy):**
    - $\pi: S \to A$
    - $\pi(s)$ 表示在状态 $s$ 时，智能体确定性地选择动作 $\pi(s)$。

2. **随机策略 (Stochastic Policy):**
    - $\pi: S \times A \to [0, 1]$
    - $\pi(a | s)$ 表示在状态 $s$ 时，智能体选择动作 $a$ 的概率。
    - 对于任意状态 $s \in S$，所有可能动作的概率之和必须为 1：$\sum_{a \in A} \pi(a | s) = 1$

最优策略（Optimal Policy）是能够使价值函数达到最优的策略。对于任何状态 $s$，最优策略 $\pi^*(s)$ 选择的动作 $a$ 应该最大化 $Q^*(s, a)$：$\pi^*(s) = \arg\max_{a \in A} Q^*(s, a)$

## 3. 术语定义：价值函数

价值函数（Value Functions）衡量了从某个状态或状态-动作对开始，遵循某个策略所能获得的期望累积折扣奖励。

1. **状态值函数 (State-Value Function):**
    - 表示从状态 $s$ 开始，遵循策略 $\pi$ 所能获得的期望累积折扣奖励：$V^\pi(s) = \mathbb{E}_\pi \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \middle| S_t = s \right]$，其中 $\mathbb{E}_\pi$ 表示在策略 $\pi$ 下的期望。
    - 也可以用贝尔曼方程（Bellman Equation）表示为：$V^\pi(s) = \sum_{a \in A} \pi(a | s) \sum_{s' \in S} P(s' | s, a) \left[ R(s, a, s') + \gamma V^\pi(s') \right]$
2. **状态-动作值函数 (Action-Value Function / Q-function):**
    - 表示在状态 $s$ 执行动作 $a$ 后，再遵循策略 $\pi$ 所能获得的期望累积折扣奖励：$Q^\pi(s, a) = \mathbb{E}_\pi \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \middle| S_t = s, A_t = a \right]$
    - 也可以用贝尔曼方程表示为：$Q^\pi(s, a) = \sum_{s' \in S} P(s' | s, a) \left[ R(s, a, s') + \gamma V^\pi(s') \right]$
    - 或者结合 $V^\pi(s')$ 和 $Q^\pi(s', a')$ 进行变换：$Q^\pi(s, a) = \sum_{s' \in S} P(s' | s, a) \left[ R(s, a, s') + \gamma \sum_{a' \in A} \pi(a' | s') Q^\pi(s', a') \right]$

最优价值函数（Optimal Value Functions）目标是找到一个最优策略 $\pi^*$，使得其价值函数达到最大。

1. **最优状态值函数:** $V^*(s) = \max_{\pi} V^\pi(s)$，它满足最优贝尔曼方程 (Bellman Optimality Equation)：$V^*(s) = \max_{a \in A} \sum_{s' \in S} P(s' | s, a) \left[ R(s, a, s') + \gamma V^*(s') \right]$
2. **最优状态-动作值函数:** $Q^*(s, a) = \max_{\pi} Q^\pi(s, a)$，它满足最优贝尔曼方程：$Q^*(s, a) = \sum_{s' \in S} P(s' | s, a) \left[ R(s, a, s') + \gamma \max_{a' \in A} Q^*(s', a') \right]$

> 贝尔曼方程（Bellman Equation）描述了价值函数的递归性质。对于任意策略 $\pi$，有：
> 
> - **状态值函数的贝尔曼方程：**$ V^\pi(s) = \sum_{a \in A} \pi(a|s) \sum_{s' \in S} P(s'|s, a) \left[ R(s, a, s') + \gamma V^\pi(s') \right]$
> - **动作值函数的贝尔曼方程：**$ Q^\pi(s, a) = \sum_{s' \in S} P(s'|s, a) \left[ R(s, a, s') + \gamma \sum_{a' \in A} \pi(a'|s') Q^\pi(s', a') \right]$
> 
> 最优贝尔曼方程（Bellman Optimality Equation）描述了在最优策略下，状态值函数和动作值函数的递归关系，是强化学习理论的核心基础。
> 
> - **最优状态值函数的贝尔曼方程：**$ V^*(s) = \max_{a \in A} \sum_{s' \in S} P(s'|s, a) \left[ R(s, a, s') + \gamma V^*(s') \right] $
> - **最优动作值函数的贝尔曼方程：**$ Q^*(s, a) = \sum_{s' \in S} P(s'|s, a) \left[ R(s, a, s') + \gamma \max_{a' \in A} Q^*(s', a') \right] $
> 
> 这些方程为动态规划、价值迭代、Q-learning 等算法提供了理论基础。

## 4. 决策指导

### 4.1. 价值导向的决策指导，Value-Based

价值导向的指导方法是指智能体通过学习每个状态（或状态-动作对）的“价值”来指导决策。其核心思想是：在每个状态下选择能够带来最大期望累积奖励的动作。常见方法包括动态规划、时序差分（TD）学习、Q-learning 等。

- **价值函数（Value Function）**：衡量从某一状态（或状态-动作对）出发，遵循某一策略所能获得的期望累积奖励。
- **策略更新**：通过不断评估和更新价值函数，智能体逐步逼近最优策略。
- **代表算法**：Q-learning、SARSA、价值迭代（Value Iteration）、策略迭代（Policy Iteration）等。

## 5. 参考文献

[Mastering Reinforcement Learning](https://gibberblot.github.io/rl-notes/index.html)